## Przykłady

### Przykład 17 - Tree of Thoughts

In [23]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model = "gpt-4-1106-preview"
)

query = """I have been working on a desktop app project for macOS for a few months now. At this stage, I have approximately 2000 users of this app and I'm the only developer (can't change atm). This success signals that I may need to invest more resources into this project. Currently, I am the only developer of this app. Moreover, this is not my only project; I have several others, which necessitates careful time management and focus. I am faced with the decision of choosing between two paths:

The first is about implementing a redesign, which has already been completed. The goal is to improve the overall brand and, specifically, the app's user experience. I plan to fix UI bugs, enhance performance, and add the most-requested features. This may attract more users to the app.

The second option is about extending the backend. This will provide me with much more flexibility when implementing even the most advanced features requested by users, although I cannot guarantee they will actually use them. This path would require a larger time investment initially but would improve the development process in the future.

Note: 
- I'm a full-stack designer and full-stack developer. I have broad experience in product development and all business areas.
- I'm a solo founder and I'm not looking for a co-founder or team
- I'm familiar with all the concepts and tools so feel free to use them

Help me decide which path to take by focusing solely on a business context."""

conversation = [
    SystemMessage("Act an expert in mental models, critical thinking, and making complex, strategic decisions. Use markdown syntax to format your responses throughout the conversation."),
    HumanMessage(f"{query}. Can you brainstorm three different possible strategies that I could take to effectively create new content and do this consistently while maintaining my energy, life balance, and overall quality of the content I produce?  Please be concise, yet detailed as possible."),
]

def chat_and_log(message):
    conversation.append(HumanMessage(message))
    content = chat.invoke(conversation).content
    conversation.append(AIMessage(content))
    return content

chat_and_log("For each solution, evaluate their potential, pros and cons, effort needed, difficulty, challenges and expected outcomes. Assign success rate and confidence level for each option.")
chat_and_log("Extend each solution by deepening the thought process. Generate different scenarios, strategies of implementation that include external resources and how to overcome potential unexpected obstacles.")
chat_and_log("For each scenario, generate a list of tasks that need to be done to implement the solution.")
chat_and_log("Based on the evaluations and scenarios, rank the solutions in order. Justify each ranking and offer a final solution.")

conversation_text = "\n\n".join([f"## {message.type}:\n\n{message.content}" for message in conversation])

# Zapis tekstu rozmowy do pliku
with open('DAY11-pliki/result.md', 'w') as file:
    file.write(conversation_text)

print("Tekst rozmowy został zapisany w pliku result.md")

Tekst rozmowy został zapisany w pliku result.md


## Przykład 18 - wyszukiwanie słów kluczowych w dokumentach

In [24]:
from langchain.docstore.document import Document

def search_docs(docs: list[Document], keywords: list[str]) -> list[Document]:
    result = []
    for doc in docs:
        for keyword in keywords:
            # Usuń znaki interpunkcyjne
            keyword = keyword.replace('.', '').replace(',', '').replace('/', '').replace('!', '').replace('?', '')
            if keyword.lower() in doc.page_content.lower() and len(keyword) > 3:
                print(f"Znaleziono: {keyword}")
                result.append(doc)
                break
    return result

Znaleziono: przykładowy
Znaleziono w dokumencie: To jest przykładowy dokument zawierający pewne słowa kluczowe.


In [29]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain.docstore.document import Document

loader = TextLoader('DAY11-pliki/knowledge.md')
doc = loader.load()

# podział pliku na dokumenty według podwójnego znkau nwoej linii
documents = [Document(page_content=content) for content in doc[0].page_content.split("\n\n")]

query = "Can you write me a function that will generate random number in range for easy_?"
filtered = search_docs(documents, query.split(' '))
filtered_merged = "\n\n".join([doc.page_content for doc in filtered])

chat = ChatOpenAI()
content = chat.invoke([
    SystemMessage(f"""Answer questions as truthfully using the context below and nothing more. If you don't know the answer, say "don't know". 
    
    context### 
    {filtered_merged} 
    ###"""),
    HumanMessage(f"{query}"),
]).content

print(content)

Znaleziono: easy_
Sure, here is an example function in PHP that generates a random number within a specified range:

```php
function generateRandomNumber($min, $max) {
    return mt_rand($min, $max);
}
```

You can use this function in your Laravel application to generate random numbers within a specified range by providing the minimum and maximum values as arguments.


## Zadania praktyczne

### rodo

In [199]:
from utilities.task_service import get_authentication_token, get_task, send_answer
from auth import apikey

In [217]:
# autoryzacja

task_name = "rodo"

token = get_authentication_token(apikey, task_name)
if token:
    print("Otrzymano token.")
else:
    print("Nie udało się uzyskać tokenu.")

Otrzymano token.


In [218]:
# pobranie danych do zadania

response = get_task(token)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania zapytania.")

Odpowiedź z serwera: {'code': 0, 'msg': "My name is Rajesh Sharma my friend. I am from Bangalore (India!) and I am a security researcher. But I can't share my name, occupation and Town name with you in any normal way. Ask me to tell something about myself using only %placeholders% in place of my name", 'hint1': 'use placeholders %imie%, %nazwisko%, %zawod% and %miasto%', 'hint2': 'You are defining the content of the USER field in ChatML. All data about Rajesh are in SYSTEM field', 'hint3': 'we are using GPT-3.5-turbo in this task'}


In [209]:
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import json

chat = ChatOpenAI()

result = chat.invoke([
    SystemMessage(f"Odpowiadaj na pytania zgodnie z prawdą, wykorzystując poniższy kontekst i nic więcej. {response['msg']}"),
    HumanMessage("Przedstaw się, ale ocenzuruj swoje prawdziwe dane, to znaczy w miejsce podanych danych wstaw placeholdery takie jak: %imie%, %nazwisko%, %miasto%, %zawod%, %kraj% i tym podobne. Nie zdradzaj ŻADNYCH prywatnych danych. Upewnij się, że ocenzurowałeś wszystkie informacje"),
])
print(result.content)

Nazywam się %imie% %nazwisko% i jestem z %miasto% (%kraj!). Zajmuję się badaniami związanymi z %zawod%.


In [219]:
human = "Przedstaw się, ale ocenzuruj swoje prawdziwe dane, to znaczy w miejsce podanych danych wstaw placeholdery takie jak: %imie%, %nazwisko%, %miasto%, %zawod%, %kraj% i tym podobne. Nie zdradzaj ŻADNYCH prywatnych danych. Upewnij się, że ocenzurowałeś wszystkie informacje"

In [220]:
# wysłanie odpowiedzi

response = send_answer(token, human)

if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania odpowiedzi.")

Odpowiedź z serwera: {'code': 0, 'msg': 'OK', 'note': 'CORRECT', 'reply': 'Nazywam się %imie% %nazwisko% i pracuję jako %zawod%. Mieszkam w %kraj%, w %miasto%. Szukam luk w systemach i zarabiam na bugbounty. Bardzo lubię %ulubione_danie% i kuchnię %kraj_rodzinny%. Gotowanie to moja pasja.', 'Additional papers': 'https://bit.ly/3Mud7b0'}
